In [1]:
%load_ext autoreload
%autoreload 2

from jupyter_cadquery import set_defaults, show, Camera, Collapse
from build123d import *
from enum import Enum
from functools import reduce
import math


set_defaults(
	viewer="Sidecar",
	edge_accuracy=0.0001,
	ticks=25,
	grid=[True, True, True],
	cad_width=780,
	axes=True,
	axes0=True,
	glass=True,
	orbit_control=True,
	reset_camera=Camera.CENTER,
	collapse=Collapse.LEAVES
)

Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList


In [2]:
WIDTH = 40
LENGTH = 145
SCREW_RADIUS = 2.5
CLEARANCE=0.2
INNER_CHAMFER = 10
OUTER_CHAMFER = 4

NUM_LIPS = 5


In [3]:
class SocketConfig(Enum):
    MALE=0
    FEMALE=1

class CenterConfig(Enum):
    CENTER=0
    LEFT=1
    RIGHT=2

class ModularMountingSegment:
    class Profiles:
        def __init__(self, radius, lip_length, tip, outer_chamfer, inner_chamfer, chamfered_radius):
            self.mount_bound = (ThreePointArc(
                (-radius, 0),
                (0, radius),
                (radius, 0)
            ) + Line(
                (radius, tip),
                (radius, 0)
            ) + Line(
                (-radius, tip),
                (-radius, 0)
            ))

            self.chamfered_bound = (ThreePointArc(
                (-chamfered_radius, 0),
                (0, chamfered_radius),
                (chamfered_radius, 0)
            ) + Line(
                (chamfered_radius, tip),
                (chamfered_radius, 0)
            ) + Line(
                (-chamfered_radius, tip),
                (-chamfered_radius, 0)
            )).wire().close()

            self.cutter_profile = (Polyline([
                (radius, -lip_length),
                (radius, tip),
                (0, -lip_length),
                (-radius, tip),
                (-radius, -lip_length)
            ]) + self.mount_bound).wire().close()

            width = radius * 2
            rect = Rectangle(width, width).wire()
            
            self.outer_profile = Face(rect.chamfer_2d(outer_chamfer, outer_chamfer, rect.vertices()))
            self.inner_profile = Face(rect.chamfer_2d(inner_chamfer, inner_chamfer, rect.vertices()))

    def __init__(self, length, width, clearance=CLEARANCE, screw_radius=SCREW_RADIUS, planar=True, sockets=(SocketConfig.MALE, SocketConfig.FEMALE)):
        self.length = length
        self.width = width
        self.clearance = clearance
        self.inner_chamfer = width / 5
        self.outer_chamfer = width / 10

        self._bar_length = self.length - (5 * width / 2)

        self._socket_pos = self._bar_length / 2
        self._chamfer_pos = self._socket_pos - 2 * self.inner_chamfer

        self.radius = self.width / 2
        self.lip_length = self.radius * 1.5
        self._tip = self.lip_length + self.radius
        self._chamfered_radius = self.radius - self.outer_chamfer

        self.profiles = self.Profiles(self.radius, self.lip_length, -self._tip, self.outer_chamfer, self.inner_chamfer, self._chamfered_radius)
        self.sockets = sockets
        self.is_planar = planar
        
        self.screw_hole_pos = self.length / 2
        self.screw_radius = screw_radius

    def _sockets(self):
        screw_cutter = Rot(90, 0, 0) * Cylinder(self.screw_radius, self.width)
        cutter = Rot(90,0,0) * Pos(0,0, -self.outer_chamfer - self.clearance / 2) * extrude(-Face(self.profiles.cutter_profile), -self.inner_chamfer - self.clearance)
        f_cutter = Pos(0, -self.inner_chamfer, 0) * cutter + Pos(0, self.inner_chamfer, 0) * cutter + screw_cutter
        m_cutter = cutter + Pos(0, 2 * self.inner_chamfer, 0) * cutter + Pos(0, -2 * self.inner_chamfer, 0) * cutter + screw_cutter

        base_sock = Rot(90, 0, 0) * loft([
            Pos(0, 0, self.radius) * Face(self.profiles.chamfered_bound),
            Pos(0, 0, self._chamfered_radius) * Face(self.profiles.mount_bound.wire().close()),
            Pos(0, 0, -self._chamfered_radius) * Face(self.profiles.mount_bound.wire().close()),
            Pos(0, 0, -self.radius) * Face(self.profiles.chamfered_bound),
        ], ruled=True) + extrude(Pos(0,0, -self.radius) * self.profiles.outer_profile, -self.radius)
        m_sock, f_sock = base_sock - m_cutter, base_sock - f_cutter

        try:
            m_sock = m_sock.fillet(self.outer_chamfer, m_sock.edges().filter_by(Axis.Y).filter_by_position(Axis.Z, -self.lip_length, -self.lip_length))
            f_sock = f_sock.fillet(self.outer_chamfer, f_sock.edges().filter_by(Axis.Y).filter_by_position(Axis.Z, -self.lip_length, -self.lip_length))

            m_sock = m_sock.fillet(self.outer_chamfer / 2, m_sock.faces().filter_by(Plane.XZ).edges() - (m_sock.edges().filter_by(Axis.X) + m_sock.edges().filter_by(Axis.Z).filter_by_position(Axis.Z, -self.width * 2, -self.width)))
            f_sock = f_sock.fillet(
                self.outer_chamfer / 2,
                f_sock.faces().filter_by(Plane.XZ).edges().filter_by_position(Axis.Y, -self.radius, self.radius, (False, False)) + f_sock.edges().filter_by_func(lambda x: x.geom_type == GeomType.CIRCLE and x.radius == self.screw_radius)
            )
        except:
            pass

        finally:
            return Pos(0,0,self._tip) * m_sock, Pos(0,0,self._tip) * f_sock

    def _bar(self):
        if self._socket_pos <= 0:
            return Part()
        
        if self._chamfer_pos <= 0:
            return loft([
                Pos(0, 0, -self._socket_pos) * self.profiles.outer_profile,
                Pos(0, 0, self._socket_pos) * self.profiles.outer_profile
            ], ruled=True)
        return loft([
            Pos(0, 0, -self._socket_pos) * self.profiles.outer_profile,
            Pos(0, 0, -self._chamfer_pos) * self.profiles.inner_profile,
            Pos(0, 0,  self._chamfer_pos) * self.profiles.inner_profile,
            Pos(0, 0,  self._socket_pos) * self.profiles.outer_profile
        ], ruled=True)

    def __repr__(self):
        return f"<{self.length=}, {self._bar_length=}, {self.width=}, {self._socket_pos=}, {self._chamfer_pos=}>"

    def construct(self):
        m, f = self._sockets()
        mapping = {SocketConfig.MALE: m, SocketConfig.FEMALE: f}
        b = Rot(90,0,0) * self._bar()
        s1 = Pos(0, -self._socket_pos, 0) * Rot(90, 0, 90) * mapping[self.sockets[1]]
        s2 = Pos(0, self._socket_pos, 0) * Rot(-90, 0, 90 if self.is_planar else 0) * mapping[self.sockets[0]]
        return b + s1 + s2
    
    def place_at(self, center: CenterConfig):
        segment = self.construct()
        if center == CenterConfig.RIGHT:
            return Pos(0, -self.screw_hole_pos, 0) * segment
        elif center == CenterConfig.LEFT:
            return Pos(0, self.screw_hole_pos, 0) * segment
        else:
            return segment
    

show(ModularMountingSegment(100, 20, CLEARANCE, SCREW_RADIUS, True).place_at(CenterConfig.LEFT))
#m = ModularMountingSegment(100, 20, CLEARANCE, SCREW_RADIUS, True).place_at(CenterConfig.LEFT)

+


In [3]:
WIDTH = 20
CHAMFER = 1
TOOTH_WIDTH = 4

import asyncio
from itertools import starmap

async def make_lip(width, tooth_thickness) -> Solid:
    profile = Polyline(
        [
            (width + tooth_thickness, -width / 2),
            (0, -width / 2),
            (0, width / 2),
            (width + tooth_thickness, width / 2)
        ]
    ) + RadiusArc(
        (width + tooth_thickness, width / 2),
        (width + tooth_thickness, -width / 2),
        width / 2
    )
    return Rot(-90, 0, 0) * extrude(Plane.XZ * Face(profile), tooth_thickness / 2, both=True) 

async def make_tooth(width, tooth_thickness) -> Solid:
    profile = Polyline(
        (tooth_thickness, -width / 2, 0),
        (0, -width / 2, 0),
        (0, width / 2, 0),
        (tooth_thickness, width / 2, 0),
        (width / 2 + tooth_thickness, 0, 0), close=True
    )
    w = Rot(-90, 0, 0) * extrude(Plane.XZ * Face(profile), tooth_thickness / 2, both=True)
    return w

async def make_socket(width, chamfer_length, is_male=True) -> Solid:
    tooth_width = width / 5
    lip, tooth = await asyncio.gather(make_lip(width, tooth_width), make_tooth(width, tooth_width))
    l = enumerate(Line((0, -width / 2 + tooth_width / 2), (0, width / 2 - tooth_width / 2)).distribute_locations(5))
    socket = reduce(lambda x,y : x + y, starmap(lambda index, pos: pos * (lip if index % 2 == is_male else tooth), l))
    return socket.chamfer(
        chamfer_length,
        chamfer_length,
        socket.faces().filter_by(Axis.Y).edges() - (
            socket.edges().filter_by_position(
                Axis.Y,
                -WIDTH / 2,
                WIDTH / 2,
                inclusive=(False, False)
            ) + socket.edges().filter_by(Axis.X)
        )
    )

async def make_male_socket(width, chamfer_length):
    socket = await make_socket(width, False)
    chamfer_edges = socket.faces().filter_by(Axis.Y).edges() - (
        socket.edges().filter_by_position(
            Axis.Y,
            -width / 2,
            width / 2,
            inclusive=(False, False)
        ) + socket.edges().filter_by(Axis.X)
    )
    return socket.chamfer(chamfer_length, chamfer_length, chamfer_edges)

async def make_female_socket(width, chamfer_length):
    socket = await make_socket(width, True)
    chamfer_edges = socket.edges()

solids : tuple[Solid, Solid] = await asyncio.gather(make_socket(WIDTH, CHAMFER, False), make_socket(WIDTH, CHAMFER, True))
female, male = solids

show(
    male.fillet(CHAMFER, male.edges().filter_by_position(Axis.Y, -WIDTH / 2 + CHAMFER, WIDTH / 2 - CHAMFER, inclusive=(False, False)).filter_by(GeomType.CIRCLE)),
    female.fillet(CHAMFER, female.edges().filter_by_position(Axis.Y, -WIDTH / 2 + CHAMFER, WIDTH / 2 - CHAMFER, inclusive=(False, False)).filter_by(GeomType.CIRCLE)),
)

++


In [5]:
lip = await make_lip(WIDTH, TOOTH_WIDTH)

loft([
    Face(Polyline(
        (-WIDTH / 2, TOOTH_WIDTH, 0),
        (-WIDTH / 2, 0, 0),
        (WIDTH / 2, 0, 0),
        (WIDTH / 2, TOOTH_WIDTH, 0),
        (0, WIDTH / 2 + TOOTH_WIDTH, 0), close=True
    )),
    Plane.XY.offset(-TOOTH_WIDTH) *Face(Polyline(
        (-WIDTH / 2 + TOOTH_WIDTH, TOOTH_WIDTH, 0),
        (-WIDTH / 2 + TOOTH_WIDTH, 0, 0),
        (WIDTH / 2 - TOOTH_WIDTH , 0, 0),
        (WIDTH / 2 - TOOTH_WIDTH , TOOTH_WIDTH, 0),
        (0, WIDTH / 2, 0), close=True
    )),
], ruled=True) + Pos(0, 0, TOOTH_WIDTH /2) * Rot(0, 0, 90) * lip

+
